In [1]:
# 미국 전체 작품 데이터 수집
import pandas as pd

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from time import sleep
import time

In [34]:
# 1. 전체 작품의 작품 이름, 장르, 하트 수 구하기
URL = 'https://www.webtoons.com/en/dailySchedule'
weeks = ["MONDAY","TUESDAY","WEDNESDAY","THURSDAY","FRIDAY","SATURDAY","SUNDAY"]

html = requests.get(URL).text
soup = BeautifulSoup(html,'html.parser')

month_titles = []; month_genres = []; month_hearts = [];

for week in weeks:
    
    ten_titles = []; ten_genres = []; ten_hearts = [];
    
    titles = soup.select(f"#dailyList > div.daily_section._list_{week} > ul > li > a > div > p.subj")
    genres = soup.select(f"#dailyList > div.daily_section._list_{week} > ul > li > a > p.genre")
    hearts = soup.select(f"#dailyList > div.daily_section._list_{week} > ul > li > a > p.grade_area > em")
    
    ten_titles = [title.text for title in titles]
    ten_genres = [genre.text for genre in genres]
    # 1.7M(str) -> 1700000(float) 으로 전처리 & ',' 제거 
    ten_hearts = [float(heart.text.replace(",","").replace("M","")) * 1000000 
                  if "M" in heart.text else float(heart.text.replace(",","")) 
                  for heart in hearts]
    
    month_titles.append(ten_titles)
    month_genres.append(ten_genres)
    month_hearts.append(ten_hearts)

In [37]:
week_genre_list = []
for week,week_genres in zip(weeks,month_genres):
    print(f"{week}: {len(week_genres)}")
    week_genre_list.append(len(week_genres))

MONDAY: 68
TUESDAY: 79
WEDNESDAY: 73
THURSDAY: 69
FRIDAY: 75
SATURDAY: 71
SUNDAY: 66


In [38]:
week_heart_list = []
for week,week_hearts in zip(weeks,month_hearts):
    print(f"{week}: {len(week_hearts)}")
    week_genre_list.append(len(week_hearts))

MONDAY: 68
TUESDAY: 79
WEDNESDAY: 73
THURSDAY: 69
FRIDAY: 75
SATURDAY: 71
SUNDAY: 66


In [32]:
week_cnt_list = []
for week,week_titles in zip(weeks,month_titles):
    print(f"{week}: {len(week_titles)}")
    week_cnt_list.append(len(week_titles))

mon: 68
tue: 79
wed: 73
thu: 69
fri: 75
sat: 71
sun: 66


In [11]:
week_cnt_list

[68, 79, 73, 69, 75, 71, 66]

In [25]:
sum(week_cnt_list)

501

In [13]:
# 2. 전체 작품의 별점 평균, 총 회차수, 조회수

driver = webdriver.Chrome('chromedriver.exe')
star_mean_list = []; ep_list = []; view_list = []; week_cnt = 0

for i,week in enumerate(weeks):
    driver.get(URL)
    time.sleep(1)
    
    week_cnt = week_cnt_list[i] #요일마다 작품 수가 다름
    
    for i in range(1,week_cnt+1):
        # 요일 별로 작품 클릭
        page = driver.find_element(by=By.CSS_SELECTOR, 
                                    value=f"#dailyList > div.daily_section._list_{week} > ul > li:nth-child({i}) > a")
    
        page.click()
        time.sleep(0.5)
                                   
        # 이동한 페이지 주소 읽고, 파싱하기
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        
        # 별점 평균 수집
        star_mean = soup.select("#_starScoreAverage")[0].getText()               
        star_mean_list.append(star_mean)
                                   
        # 조회수 수집
        view = soup.select("#_asideDetail > ul > li:nth-child(1) > em")[0].getText()   
        view_list.append(view)
                                   
        # 회차수 수집
        ep = soup.select("#_listUl > li:nth-child(1) > a > span.tx")[0].getText()   
        ep_list.append(ep)
        
        driver.back()                        

C:\Users\Jung_a\AppData\Local\Temp/ipykernel_1124/732188065.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [23]:
len(star_mean_list)

501

In [16]:
ep_list[:5]

['#17', '#6', '#54', '#31', '#69']

In [27]:
len(ep_list)

501

In [17]:
view_list[:5]

['20.7M', '2.1M', '81.5M', '12.9M', '30.4M']

In [28]:
len(view_list)

501

In [18]:
# 3. row & column 제작
## 3.1  [제목, 요일, 순위] row data 제작

row_data = []
weeks = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
week = ""
rank = 0
titles = ""

for i, week_titles in enumerate(month_titles):
    week = weeks[i]
    for j, title in enumerate(week_titles):
        rank = j + 1
        row_data.append([title,week,rank])

In [26]:
len(row_data)

501

In [39]:
## 3.2 장르, 하트수 칼럼 제작

genres = [genre for week_genres in month_genres for genre in week_genres ]
hearts = [heart for week_hearts in month_hearts for heart in week_hearts]

In [ ]:
len(genres)

501

In [20]:
## 3.3 별점 평균, 회차수, 조회수 전처리한 칼럼 제작

star_means = [float(star_mean) for star_mean in star_mean_list]
total_ep = [int(ep.replace("#","")) for ep in ep_list]
views = []
for view in view_list:
    view = view.replace(",","")
    if "M" in view:
        view = float(view.replace("M","")) * 1000000
    elif "B" in view:
        view = float(view.replace("B","")) * 1000000000 
    else:
        view = float(view)
    views.append(round(view,1))

In [40]:
# 4. 데이터프레임 제작

df = pd.DataFrame(row_data,columns = ['title', 'week','rank'])
df["genre"] = genres
df["heart"] = hearts
df["star_mean"] = star_means
df["ep"] = total_ep
df["view"] = views

In [41]:
df

,title,week,rank,genre,heart,star_mean,ep,view
0,Marry My Husband,mon,1,Romance,1700000.0,9.63,17,20700000.0
1,I’m the Queen in This Life,mon,2,Fantasy,211822.0,9.53,6,2100000.0
2,Everything is Fine,mon,3,Horror,5200000.0,9.74,54,81500000.0
3,Blood Reverie,mon,4,Romance,1300000.0,9.25,31,12900000.0
4,To The Stars and Back,mon,5,Slice of life,3400000.0,9.66,69,30400000.0
...,...,...,...,...,...,...,...,...
496,The Remarried Empress,sun,62,Fantasy,25300000.0,9.86,125,285100000.0
497,R-X: RAYER HUNTER,sun,63,Sci-fi,97264.0,8.15,40,1000000.0
498,Blades of Furry,sun,64,Romance,3200000.0,9.58,62,31200000.0
499,Everywhere & Nowhere,sun,65,Comedy,11300000.0,9.57,234,98400000.0


In [44]:
# 5. 중복 제거
## 5.1 중복 칼럼 확인
df[df.duplicated(['title'])]

,title,week,rank,genre,heart,star_mean,ep,view
153,My Giant Nerd Boyfriend,wed,7,Slice of life,62400000.0,9.73,800,744700000.0
162,Scroll if You Dare,wed,16,Horror,548965.0,9.54,27,6300000.0
164,Safely Endangered,wed,18,Comedy,23300000.0,9.56,837,397800000.0
225,To The Stars and Back,thu,6,Slice of life,3400000.0,9.66,69,30400000.0
244,Ingrid the Plague Doctor,thu,25,Slice of life,423226.0,9.44,46,3600000.0
245,Finding Fiends,thu,26,Comedy,1200000.0,9.76,98,7400000.0
271,Gecko Chan,thu,52,Comedy,503304.0,9.15,124,3400000.0
274,Discovering Love: A Harlequin Romance Collection,thu,55,Romance,147093.0,8.90,94,1300000.0
292,Vampire Husband,fri,4,Slice of life,364561.0,9.56,11,2600000.0
296,1HP Club,fri,8,Fantasy,4000000.0,9.64,95,34300000.0


In [45]:
# 중복 데이터프레임 개수
print(len(df[df.duplicated(['title'])]))

27


In [47]:
## 5.2 중복 칼럼 제거

df_undp = df.drop_duplicates(['title'],ignore_index = True)

In [48]:
df_undp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      474 non-null    object 
 1   week       474 non-null    object 
 2   rank       474 non-null    int64  
 3   genre      474 non-null    object 
 4   heart      474 non-null    float64
 5   star_mean  474 non-null    float64
 6   ep         474 non-null    int64  
 7   view       474 non-null    float64
dtypes: float64(3), int64(2), object(3)
memory usage: 29.8+ KB


In [49]:
df_undp.shape

(474, 8)

In [50]:
# 6 저장
df_undp.to_excel('america.xlsx')

In [51]:
df_undp.to_csv('america.csv')

In [2]:
# etc1. 구글 제목 자동 검색 코드 (소설 원작 여부 검색 시 빠르게 하기 위해 사용)
df_undp = pd.read_csv("data/america.csv")

df_dict = df_undp.to_dict()
dic_val = df_dict["title"].values()
dic_list = list(dic_val)

browser = webdriver.Chrome()
browser.get('https://www.google.com/search?q=0The+Lazy+Lord+Masters+the+Sword&rlz=1C1CHZN_koKR1021KR1021&biw=763&bih=646&sxsrf=ALiCzsbVZx7IkMYKQCDF4KsGdlExwwk8eg%3A1671265709315&ei=rX2dY63yEtuohwPmopuADQ&ved=0ahUKEwjt_Lb3nYD8AhVb1GEKHWbRBtAQ4dUDCA8&oq=0The+Lazy+Lord+Masters+the+Sword&gs_lcp=Cgxnd3Mtd2l6LXNlcnAQDDIHCCEQoAEQCjIHCCEQoAEQCjIHCCEQoAEQCjoHCCMQ6gIQJzoECCMQJzoICAAQgAQQsQM6BQguEIAEOgsIABCABBCxAxCDAToECAAQA0oECEEYAEoECEYYAEoECEEYAEoECEYYAEoECEEYAEoECEYYAFDtM1jlZWC8bGgMcAF4AIABoAGIAacCkgEDMC4ymAEAoAEBoAECsAEJwAEB&sclient=gws-wiz-serp')

for title in dic_list:
    
    # 지우기
    elem = browser.find_element(by=By.CSS_SELECTOR, value="#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.dRYYxd > div.BKRPef > div > span > svg")
    elem.click()
    
    # 검색어 쓰기
    
    elem = browser.find_element(by=By.CSS_SELECTOR, value="#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > div > div.a4bIc > input") 
    elem.send_keys(title)
    
    # 돋보기  클릭
    elem = browser.find_element(by=By.CSS_SELECTOR, value="#tsf > div:nth-child(1) > div.A8SBwf > div.RNNXgb > button > div > span > svg")
    elem.click()
    
    time.sleep(5)
    

    
    #browser.back()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x0038F243]
	(No symbol) [0x00317FD1]
	(No symbol) [0x0020D04D]
	(No symbol) [0x001F2D7A]
	(No symbol) [0x0025BE7B]
	(No symbol) [0x0026C196]
	(No symbol) [0x00258386]
	(No symbol) [0x0023163C]
	(No symbol) [0x0023269D]
	GetHandleVerifier [0x00629A22+2655074]
	GetHandleVerifier [0x0061CA24+2601828]
	GetHandleVerifier [0x00438C0A+619850]
	GetHandleVerifier [0x00437830+614768]
	(No symbol) [0x003205FC]
	(No symbol) [0x00325968]
	(No symbol) [0x00325A55]
	(No symbol) [0x0033051B]
	BaseThreadInitThunk [0x76DE00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77557BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77557B8E+238]


In [12]:
# etc2. 제목 자동 검색 코드 (소설 원작 여부 검색 시 빠르게 하기 위해 사용)
df_undp = pd.read_csv("data/korea.csv")

df_dict = df_undp[23:].to_dict()
dic_val = df_dict["title"].values()
dic_list = list(dic_val)

browser = webdriver.Chrome()
#browser.get('https://webtoon.fandom.com/wiki/Special:Search?query=+&scope=internal&contentType=&ns%5B0%5D=0&ns%5B1%5D=2900')

for title in dic_list:
    # 되돌아가기
    browser.get('https://webtoon.fandom.com/wiki/Special:Search?query=+&scope=internal&contentType=&ns%5B0%5D=0&ns%5B1%5D=2900')
    time.sleep(2)
    
    # 검색어 쓰기
    elem = browser.find_element(by=By.CLASS_NAME, value="unified-search__input__query") 
    elem.send_keys(title)
    
    # 돋보기  클릭
    elem = browser.find_element(by=By.CLASS_NAME, value="wds-button.unified-search__input__button")
    elem.click()
    
    time.sleep(3)
    
    

ElementClickInterceptedException: Message: element click intercepted: Element <button type="submit" class="wds-button unified-search__input__button" title="Search">...</button> is not clickable at point (827, 52). Other element would receive the click: <iframe frameborder="0" src="https://1f8623e768ee0490543c5916bd98125a.safeframe.googlesyndication.com/safeframe/1-0-40/html/container.html" id="google_ads_iframe_/5441/wka1b.LB/top_leaderboard/desktop/ucp_desktop-fandom-search/_not_a_top1k_wiki-ent_0" title="3rd party ad content" name="" scrolling="no" marginwidth="0" marginheight="0" width="728" height="90" data-is-safeframe="true" sandbox="allow-forms allow-popups allow-popups-to-escape-sandbox allow-same-origin allow-scripts allow-top-navigation-by-user-activation" role="region" aria-label="Advertisement" tabindex="0" data-google-container-id="1" style="border: 0px; vertical-align: bottom; width: 728px; height: 90px;" data-load-complete="true"></iframe>
  (Session info: chrome=108.0.5